<center>
    <img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width="1000" alt="cognitiveclass.ai logo"  />
</center>

# Assignment: Exploratory Data Analysis with SQL

Estimated time needed: 60 minutes

# Introduction

Using this R notebook you will perform exploratory data analysis tasks using SQL queries with the RODBC R package. You will not be graded here, but you will be asked to present some of these results in your final assignment, where you will be graded on the accuracy of your results as well as the content of your SQL queries.


### Establish your Db2 connection

Load the 'RODBC' library, and use the 'odbcConnect( )' function as you did in the previous lab to establish the connection to your Db2 assets.\
Provided you successfully loaded your data into the tables in that lab, you are now ready to start running SQL queries using the RODBC library as you did in Course 3.


In [1]:
# provide your solution here
library(RODBC)

#Create a database connection
dsn_driver <- "{IBM DB2 ODBC Driver}"
dsn_database <- "bludb"            
dsn_hostname <- "55fbc997-9266-4331-afd3-888b05e734c0.bs2io90l08kqb1od8lcg.databases.appdomain.cloud"  
dsn_port <- "31929"               
dsn_protocol <- "TCPIP"           
dsn_uid <- "gmx74666"        
dsn_pwd <- "0L7nXAM2OKo5TGZs"
dsn_security <- "ssl"

#Create a connection string and connect to the database
conn_path <- paste("DRIVER=",dsn_driver,
                  ";DATABASE=",dsn_database,
                  ";HOSTNAME=",dsn_hostname,
                  ";PORT=",dsn_port,
                  ";PROTOCOL=",dsn_protocol,
                  ";UID=",dsn_uid,
                  ";PWD=",dsn_pwd,
                  ";SECURITY=",dsn_security,        
                    sep="")

conn_path
conn <- odbcDriverConnect(conn_path)


[1] "DRIVER={IBM DB2 ODBC Driver};DATABASE=bludb;HOSTNAME=55fbc997-9266-4331-afd3-888b05e734c0.bs2io90l08kqb1od8lcg.databases.appdomain.cloud;PORT=31929;PROTOCOL=TCPIP;UID=gmx74666;PWD=0L7nXAM2OKo5TGZs;SECURITY=ssl"

In [2]:
#sql.info <- sqlTypeInfo(conn) #database information about the data types selected
conn.info <- odbcGetInfo(conn) #driver information

sql.info <- sqlTypeInfo(conn)
conn.info <- odbcGetInfo(conn)
conn.info

DBMS_Name         DBMS_Ver  Driver_ODBC_Ver Data_Source_Name 
"DB2/LINUXX8664"     "11.05.0600"          "03.51"               "" 
     Driver_Name       Driver_Ver         ODBC_Ver      Server_Name 
      "libdb2.a"     "11.05.0600"          "03.52"            "DB2"

## Task 1 - Record Count

#### Determine how many records are in the seoul_bike_sharing dataset.


### Solution 1


In [3]:
# provide your solution here
# provide your solution here
query <- paste("SELECT COUNT(*) AS COUNT FROM SEOUL_BIKE_SHARING")
seoul_bike_count <- sqlQuery(conn, query)
seoul_bike_count

,COUNT
,<int>
1,8465


## Task 2 - Operational Hours

#### Determine how many hours had non-zero rented bike count.


### Solution 2


In [4]:
# provide your solution here
query2 <- paste("SELECT SUM(HOUR) FROM SEOUL_BIKE_SHARING WHERE RENTED_BIKE_COUNT<>0")
total_hrs <- sqlQuery(conn, query2)
total_hrs

,1
,<int>
1,97407


## Task 3 - Weather Outlook

#### Query the the weather forecast for Seoul over the next 3 hours.

Recall that the records in the CITIES_WEATHER_FORECAST dataset are 3 hours apart, so we just need the first record from the query.


### Solution 3


In [5]:
# provide your solution here
query3 <- paste("SELECT * FROM SEOUL_BIKE_SHARING LIMIT(1)")
next_3hr <- sqlQuery(conn, query3)
next_3hr

,DATE,RENTED_BIKE_COUNT,HOUR,TEMPERATURE,HUMIDITY,WIND_SPEED,VISIBILITY,DEW_POINT_TEMPERATURE,SOLAR_RADIATION,RAINFALL,SNOWFALL,SEASONS,HOLIDAY,FUNCTIONING_DAY
,<fct>,<int>,<int>,<dbl>,<int>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<fct>,<fct>
1,01/12/2017,254,0,-5.2,37,2.2,2000,-17.6,0,0,0,Winter,No Holiday,Yes


## Task 4 - Seasons

#### Find which seasons are included in the seoul bike sharing dataset.


### Solution 4


In [6]:
# provide your solution here
query4 <- paste("SELECT DISTINCT SEASONS FROM SEOUL_BIKE_SHARING")
distinct_seasons <- sqlQuery(conn, query4)
distinct_seasons

,SEASONS
,<fct>
1,Autumn
2,Spring
3,Summer
4,Winter


## Task 5 - Date Range

#### Find the first and last dates in the Seoul Bike Sharing dataset.


### Solution 5


In [7]:
# provide your solution here
query5 <- print("SELECT MIN(DATE) AS LAST_DT, MAX(DATE) AS FIRST_DT FROM SEOUL_BIKE_SHARING")
first_lastDT <- sqlQuery(conn, query5)
first_lastDT

[1] "SELECT MIN(DATE) AS LAST_DT, MAX(DATE) AS FIRST_DT FROM SEOUL_BIKE_SHARING"


,LAST_DT,FIRST_DT
,<fct>,<fct>
1,01/01/2018,31/12/2017


## Task 6 - Subquery - 'all-time high'

#### determine which date and hour had the most bike rentals.


### Solution 6


In [36]:
# provide your solution here
query6 <- paste("SELECT DATE, SEASONS, HOUR, RENTED_BIKE_COUNT FROM SEOUL_BIKE_SHARING 
                WHERE RENTED_BIKE_COUNT = (SELECT MAX(RENTED_BIKE_COUNT) FROM SEOUL_BIKE_SHARING)")
most_bike_rentals <- sqlQuery(conn, query6)
most_bike_rentals

,DATE,SEASONS,HOUR,RENTED_BIKE_COUNT
,<fct>,<fct>,<int>,<int>
1,19/06/2018,Summer,18,3556


## Task 7 - Hourly popularity and temperature by season

#### Determine the average hourly temperature and the average number of bike rentals per hour over each season. List the top ten results by average bike count.


### Solution 7


In [9]:
# provide your solution here
query7 <- paste("SELECT HOUR, SEASONS, AVG(TEMPERATURE) AS AVG_TEMP, AVG(RENTED_BIKE_COUNT) AS AVG_NO_BIKES FROM SEOUL_BIKE_SHARING 
                GROUP BY HOUR, SEASONS
                ORDER BY AVG_NO_BIKES DESC                
                LIMIT 10")
popularity_by_season <- sqlQuery(conn, query7)
popularity_by_season

,HOUR,SEASONS,AVG_TEMP,AVG_NO_BIKES
,<int>,<fct>,<dbl>,<int>
1,18,Summer,29.38696,2135
2,18,Autumn,16.03086,1983
3,19,Summer,28.27283,1889
4,20,Summer,27.06630,1801
5,21,Summer,26.27826,1754
6,18,Spring,15.97222,1689
7,22,Summer,25.69891,1567
8,17,Autumn,17.27778,1562
9,17,Summer,30.07500,1526


## Task 8 - Rental Seasonality

#### Find the average hourly bike count during each season.

Also include the minimum, maximum, and standard deviation of the hourly bike count for each season.


### Solution 8


In [12]:
# provide your solution here
query8 <- paste("SELECT SEASONS, AVG(RENTED_BIKE_COUNT) AS AVG_COUNT, MIN(RENTED_BIKE_COUNT) AS MIN_COUNT, MAX(RENTED_BIKE_COUNT) AS MAX_COUNT,
                STDDEV(RENTED_BIKE_COUNT) AS STN_DEV FROM SEOUL_BIKE_SHARING
                GROUP BY SEASONS
                ORDER BY AVG_COUNT DESC")

seasonal_hourly_bike_count <- sqlQuery(conn, query8)
seasonal_hourly_bike_count

,SEASONS,AVG_COUNT,MIN_COUNT,MAX_COUNT,STN_DEV
,<fct>,<int>,<int>,<int>,<dbl>
1,Summer,1034,9,3556,690.0884
2,Autumn,924,2,3298,617.3885
3,Spring,746,2,3251,618.5247
4,Winter,225,3,937,150.3374


Let's explore a bit and see what might be the most significant contributing factors in terms of the provided data.


## Task 9 - Weather Seasonality

#### Consider the weather over each season. On average, what were the TEMPERATURE, HUMIDITY, WIND_SPEED, VISIBILITY, DEW_POINT_TEMPERATURE, SOLAR_RADIATION, RAINFALL, and SNOWFALL per season?

Include the average bike count as well , and rank the results by average bike count so you can see if it is correlated with the weather at all.


### Solution 9


In [13]:
# provide your solution here
query9 <- paste("SELECT SEASONS, AVG(TEMPERATURE) AS TEMP, AVG(HUMIDITY) AS HUMIDITY, AVG(WIND_SPEED) AS WIND_SPEED, AVG(VISIBILITY) AS VISIBILITY,
                AVG(DEW_POINT_TEMPERATURE) AS DEWPOINT_TEMP, AVG(SOLAR_RADIATION) AS SOLAR_RADIATION, AVG(RAINFALL) AS RAINFALL, AVG(RENTED_BIKE_COUNT) AS BIKE_COUNT FROM SEOUL_BIKE_SHARING
                GROUP BY SEASONS
                ORDER BY AVG(RENTED_BIKE_COUNT) DESC")
weather_seasonality <- sqlQuery(conn, query9)
weather_seasonality

,SEASONS,TEMP,HUMIDITY,WIND_SPEED,VISIBILITY,DEWPOINT_TEMP,SOLAR_RADIATION,RAINFALL,BIKE_COUNT
,<fct>,<dbl>,<int>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<int>
1,Summer,26.587274,64,1.609420,1501,18.750136,0.7612545,0.25348732,1034
2,Autumn,13.821167,59,1.492101,1558,5.150594,0.5227827,0.11765617,924
3,Spring,13.021389,58,1.857778,1240,4.091389,0.6803009,0.18694444,746
4,Winter,-2.540463,49,1.922685,1445,-12.416667,0.2981806,0.03282407,225


## Task 10 - Total Bike Count and City Info for Seoul

#### Use an implicit join across the WORLD_CITIES and the BIKE_SHARING_SYSTEMS tables to determine the total number of bikes avaialble in Seoul, plus the following city information about Seoul: CITY, COUNTRY, LAT, LON, POPULATION, in a single view.

Notice that in this case, the CITY column will work for the WORLD_CITIES table, but in general you would have to use the CITY_ASCII column.


### Solution 10


In [15]:
# provide your solution here
query10 <- paste("SELECT WC.CITY_ASCII, WC.COUNTRY, WC.LAT, WC.LNG, WC.POPULATION, BSS.BICYCLES FROM WORLD_CITIES AS WC, BIKE_SHARING_SYSTEMS AS BSS
                WHERE WC.CITY_ASCII = BSS.CITY")
total_bike_city_info <- sqlQuery(conn, query10)
total_bike_city_info

,CITY_ASCII,COUNTRY,LAT,LNG,POPULATION,BICYCLES
,<fct>,<fct>,<dbl>,<dbl>,<int>,<int>
1,Mendoza,Argentina,-32.88,-68.83,114822,40
2,Mumbai,India,18.96,72.83,23355000,NA
3,Shanghai,China,31.16,121.46,22120000,19165
4,Seoul,"Korea, South",37.58,127.00,21794000,20000
5,Mexico City,Mexico,19.43,-99.13,20996000,6500
6,Guangzhou,China,23.12,113.25,20902000,4840
7,Beijing,China,39.90,116.39,19433000,16000
8,Moscow,Russia,55.75,37.61,17125000,4300
9,Bangkok,Thailand,13.75,100.51,17066000,NA


## Task 11 - Find all city names and coordinates with comparable bike scale to Seoul's bike sharing system

#### Find all cities with total bike counts between 15000 and 20000. Return the city and country names, plus the coordinates (LAT, LNG), population, and number of bicycles for each city.

Later we will ask you to visualize these similar cities on leaflet, with some weather data.


### Solution 11


In [35]:
# provide your solution here

query11 <- paste("SELECT WC.CITY_ASCII, WC.COUNTRY, WC.LAT, WC.LNG, WC.POPULATION, BSS.BICYCLES FROM WORLD_CITIES AS WC, BIKE_SHARING_SYSTEMS AS BSS
                WHERE BSS.CITY = WC.CITY_ASCII AND BSS.BICYCLES LIKE '15___' OR BSS.BICYCLES LIKE '16___' OR BSS.BICYCLES LIKE '17___' OR BSS.BICYCLES LIKE '18___' OR BSS.BICYCLES LIKE '19___'")
total_bike_15000_info <- sqlQuery(conn, query11)
total_bike_15000_info

,CITY_ASCII,COUNTRY,LAT,LNG,POPULATION,BICYCLES
,<fct>,<fct>,<dbl>,<dbl>,<int>,<int>
1,Tokyo,Japan,35.68,139.69,37977000,16000
2,Jakarta,Indonesia,-6.21,106.84,34540000,16000
3,Delhi,India,28.66,77.23,29617000,16000
4,Mumbai,India,18.96,72.83,23355000,16000
5,Manila,Philippines,14.59,120.97,23088000,16000
6,Shanghai,China,31.16,121.46,22120000,16000
7,Sao Paulo,Brazil,-23.55,-46.63,22046000,16000
8,Seoul,"Korea, South",37.58,127.00,21794000,16000
9,Mexico City,Mexico,19.43,-99.13,20996000,16000


In [ ]:
close(conn)

## Author(s)

<h4> Jeff Grossman </h4>

## Contributor(s)

<h4> Yan Luo, Rav Ahuja </h4>

## Change log

| Date       | Version | Changed by    | Change Description                  |
| ---------- | ------- | ------------- | ----------------------------------- |
| 2021-05-04 | 0.2     | Jeff Grossman | Finalize                            |
| 2021-04-22 | 0.2     | Jeff Grossman | Update as per peer review           |
| 2021-04-10 | 0.2     | Jeff Grossman | Add data sources and implicit joins |
| 2021-04-05 | 0.1     | Jeff Grossman | Start content creation              |

## <h3 align="center"> © IBM Corporation 2021. All rights reserved. <h3/>
